In [304]:
# ! pip install smart_open
# ! pip install emoji
# ! pip install openpyxl
# ! pip install ipywidgets

! conda install xeus-python  -c conda-forge  
# conda activate jupyterlab-debugger

^C


In [1]:
import re
import os
import sys
import pandas as pd
import logging

from IPython.display import HTML, display

import pandas as pd
import sklearn

import wandb

import smart_open
import numpy as np 
from sklearn import preprocessing

import emoji

from simpletransformers.classification import ClassificationModel, ClassificationArgs
from simpletransformers.classification import ClassificationModel
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()


pd.set_option('display.max_colwidth', 0)



from sklearn.model_selection import train_test_split



pd.set_option('display.max_colwidth', 0)

from tqdm import tqdm_notebook

In [2]:
from tqdm.auto import tqdm
from time import sleep

for i in tqdm(range(50)):
    sleep(0.05)

  0%|          | 0/50 [00:00<?, ?it/s]

# Create training / test set 

In [51]:
def clean_twts(tw):

  # remove urls
  tw = str(tw)
  pattern = 'https{0,1}:\/\/t.co\/[a-zA-Z0-9]+'
  tw = re.sub(pattern, "", tw)

  # remove @
  pattern = '@[a-zA-Z0-9_]+ '
  tw = re.sub(pattern, "", tw)

  # remove emojis
  tw = emoji.demojize(tw)

  return tw

def clean_twt_row(row):
  return clean_twts(row['message'])

# all_sample_file = r'https://raw.githubusercontent.com/gladcolor/tweet_classification/master/samples_8400_demojized.zip'

# samples_df = pd.read_csv(all_sample_file)

def get_positive_sample():
    train_sample_file = r'/content/drive/MyDrive/tweet_classification/d_train.xlsx'
    train_samples_df = pd.read_excel(train_sample_file)

    test_sample_file = r'/content/drive/MyDrive/tweet_classification/d_test.xlsx'
    test_samples_df = pd.read_excel(test_sample_file)
    
    return train_samples_df, test_samples

# test_sample_file = r'/content/drive/MyDrive/tweet_classification/suicide_twts339k.zip'
# test_samples_df = pd.read_csv(test_sample_file, engine='c', encoding='utf16')


# remove emojis
# train_samples_df['message'] = train_samples_df['message'].apply(emoji.demojize)
# test_samples_df['message'] = test_samples_df['message'].apply(emoji.demojize)

def _correct_label(df, correct_df, text_col="message", label_col="Correct_label"):    
    merged_df = df.merge(correct_df, left_on='message', right_on='message', how='left')
    print("Need to correct tweet count:", len(df))
    mask_idx = merged_df['Correct_label'].isna() 
    mask_idx = (~mask_idx) & (merged_df['Correct_label'] != merged_df['Label'])
    mask_idx = ~ mask_idx
    print("Matched tweet count:", (~mask_idx).sum())
    
    # merged_df['Difference'] = ""
    # merged_df.loc[~mask_idx, 'Difference'] = (merged_df[~mask_idx]['Label'].astype(str) + "=>" +  merged_df[~mask_idx]['Correct_label'].astype(str)).to_list()
    merged_df.loc[~mask_idx, 'Label'] = merged_df[~mask_idx]['Correct_label'].to_list()
    

    return merged_df

def correct_label(df):
    df1 = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\bi_label_in_test_correct_labels.csv')
    df2 = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\bi_label_in_train_correct_labels.csv')
    correct_df = pd.concat([df1, df2])
    correct_df = pd.concat([df1, df2]).drop_duplicates()
    correct_df['message'] = correct_df.apply(clean_twt_row, axis=1)
    correct_df = correct_df.drop_duplicates()
    # correct_df = correct_df.rename(columns={'Label': "Correct_label"})
    
    # return correct_df
    
    df = _correct_label(df, correct_df=correct_df, text_col="message", label_col="Correct_label")
    
    df = df.drop(columns=['Correct_label'])
    
    
    return df


def get_train_sample_Japanese():    
    # positive tweet only
    # samples_df1 = pd.read_csv(r'/content/drive/Shareddrives/T5/Japan_Tweets_2021_01_01_2022_01_01/week2_positive1083.csv')
    samples_df1 = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\week2_positive1083.csv')  # count: 1083
    

    # has positive and negative
    # samples_df2 = pd.read_csv(r'/content/drive/Shareddrives/T5/Japan_Tweets_2021_01_01_2022_01_01/「１」LIZ_2021-12-10T12_49_21.000Z_9492.csv')
    samples_df2 = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\「１」LIZ_2021-12-10T12_49_21.000Z_9492.csv')  # positive count: 1061

    # positive only
    # samples_df3 = pd.read_csv(r'/content/drive/Shareddrives/T5/Japan_Tweets_2021_01_01_2022_01_01/positive_5087.csv')
    samples_df3 = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\positive_5087.csv')  # count: 5087

    # all negative
    # samples_df4 = pd.read_excel(r'/content/drive/Shareddrives/T5/Japan_Tweets_2021_01_01_2022_01_01/TotalNegative.xlsx')
    samples_df4 = pd.read_excel(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\TotalNegative.xlsx')

    # positive
    # samples_df5 = pd.read_excel(r'/content/drive/Shareddrives/T5/Japan_Tweets_2021_01_01_2022_01_01/SuicideRelate_Round2_BasedOn20000.xlsx')
    samples_df5 = pd.read_excel(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\SuicideRelate_Round2_BasedOn20000.xlsx')   # count: 90

    # negative from LIZ's second' file
    # samples_df6 = pd.read_csv(r'/content/drive/Shareddrives/T5/Japan_Tweets_2021_01_01_2022_01_01/LIZ_negative_2021-12-20T11_42_05.000Z_2021-12-21T02_34_19.000Z_9385.csv')
    samples_df6 = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\LIZ_negative_2021-12-20T11_42_05.000Z_2021-12-21T02_34_19.000Z_9385.csv')   # count: 
    samples_df6 = samples_df6[['message']]
    
    samples_df7 = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\SuicideRelate_negative_Round2_BasedOn20000.csv')
    # samples_df7 = samples_df7.sample(n=7000 , random_state=42)  # n=7000, means about a half of non-"自殺" tweets
    
    samples_df8 = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\zhenye_negative.csv')  # count: 12000 without RT 
       
    # positive in samples_df2
    positive_idx = samples_df2['RISK(YES:1,NO:0)'] == 1
    samples_df2_positive = samples_df2[positive_idx][['id', 'text']]
    samples_df2_positive['message'] = samples_df2_positive['text']
    samples_df2_positive = samples_df2_positive.drop(columns=['text', 'id'])

    # negative in samples_df2
    samples_df2_negative = samples_df2[~positive_idx][['id', 'text']]
    samples_df2_negative['message'] = samples_df2_negative['text']
    samples_df2_negative = samples_df2_negative.drop(columns=['text', 'id'])    
    
    samples_df4['message'] = samples_df4['CONTENT']
    samples_df4 = samples_df4.drop(columns=['CONTENT', 'TWEETID'])

    samples_df5 = samples_df5[['message']]

    positive_df = pd.concat([samples_df1, samples_df2_positive, samples_df3, samples_df5]).sample(frac=1, random_state=42)  # samples_df1, samples_df2_positive, ,
    negative_df = pd.concat([samples_df2_negative, samples_df6, samples_df8]).dropna().sample(frac=1, random_state=42)   # samples_df4, samples_df2_negative, samples_df6, samples_df7: 19900
    positive_df['Label'] = 1
    negative_df['Label'] = 0    
    positive_df = positive_df[positive_df['message'].str[:2] != "RT"]
    negative_df = negative_df[negative_df['message'].str[:2] != "RT"]
    
    positive_df['message'] = positive_df.apply(clean_twt_row, axis=1)
    negative_df['message'] = negative_df.apply(clean_twt_row, axis=1)
    
    all_samples_df = pd.concat([positive_df, negative_df]) 
    
    print("Positive sample count before correction:", len(positive_df))
    print("Negative sample count before correction:", len(negative_df))
    print("Total sample count beforoe correction:", len(all_samples_df))
    
    # Correct the wrong labels.
    # positive_df = correct_label(positive_df)
    # negative_df = correct_label(negative_df)
    
    all_samples_df = correct_label(all_samples_df)
    # remove the "自殺" tweets
    keywords = ['首吊り', '首を吊る', '首つり', '死ぬ気', '自分を傷つける', 'この世を去る', '死ぬに値する', '自分の人生を終わらせたいという願望', '死にたい', '自傷', '私の命を奪う', '死にたい', '死にたいです', '私の遺書', '私の人生を終わらせる', '決して起きない', '生きる価値がない', '飛び降りる', '永遠に眠る', '電車に飛び込む', '私がいないほうがいい', '生きるのに疲れた', '一人で死ぬ', '永遠に眠る', '私の悲しい人生', 'ストレスを感じる', 'ストレスで参っている', '感情の起伏が激しい', '私自身が嫌い', '精神的に弱い', '練炭', '焼身', '服毒', 'もう死にたい', '自殺サイト楽に死ねる方法', '生きることがつらい', '死にたい 助けて', '安楽死方法', '一番楽に死ねる方法', '簡単に死ねる方法', '消えたい', '確実に死ねる方法', '生きる意味が分からない', 'うつ 死にたい']
    pattern = '|'.join(keywords)  #"自殺"  "自殺", 
    mask_ids = all_samples_df['message'].str.contains(pattern)#.sum()
    # all_samples_df = all_samples_df[mask_ids]
    # print("Total sample count after removing 自殺 tweets:", len(all_samples_df))
    
    
    positive_df = all_samples_df.query("Label == 1") 
    negative_df = all_samples_df.query("Label == 0")
    
    positive_df = positive_df.sample(frac=1, random_state=42) # should be 42
    negative_df = negative_df.sample(frac=1, random_state=42)
    
    print("Positive sample count after correction:", len(positive_df))
    print("Negative sample count after correction:", len(negative_df))
    print("Total sample count after correction:", len(all_samples_df))

 

    
    test_positive_ratio = 0.15  / 0.85  
    positive_test_ratio = 0.2  
    test_positive_cnt = int(len(positive_df) * positive_test_ratio)
    test_negative_cnt = int(test_positive_cnt / test_positive_ratio)   
    
    print("test_positive_cnt, test_negative_cnt:", test_positive_cnt, test_negative_cnt)

    test_positive_df = positive_df.iloc[:test_positive_cnt]
    test_negative_df = negative_df.iloc[:test_negative_cnt]

    train_positive_df = positive_df.iloc[test_positive_cnt:]
    train_negative_df = negative_df.iloc[test_negative_cnt:]
    
    
    test_df = pd.concat([test_positive_df, test_negative_df]).sample(frac=1, random_state=42)
    train_df = pd.concat([train_positive_df, train_negative_df]).sample(frac=1, random_state=42)
 
    # print("Negative count int natural_dist_positive_df:", test_positive_cnt)
    # print("Negative count int natural_dist_negative_df no duplicates:", len(natural_dist_positive_df.drop_duplicates(subset='message')))

    print("\nPositive count:", len(positive_df))
    print("Positive count no duplicates:", len(positive_df.drop_duplicates(subset='message')))

    print("\nNegative count:", len(negative_df))
    print("Negative count no duplicates:", len(negative_df.drop_duplicates(subset='message')))

    print("\nTraining set sample count (before over-sampling positive tweets):", len(train_df))
    print("Training set positive count:", len(train_df.query("Label == 1")))
    print("Training set negative count:", len(train_df.query("Label == 0")))
    
    # over sampling     
    train_positive_df = train_positive_df.sample(n=int(len(train_negative_df) * 3), replace=True, random_state=42)  # Note when replace=True, *1 will not return all samples!
    train_df = pd.concat([train_positive_df, train_negative_df]).sample(frac=1, random_state=42)

    print("Train set positive count no duplicates:", len(train_positive_df.drop_duplicates(subset='message')))
    print("Train set negative count no duplicates:", len(train_negative_df.drop_duplicates(subset='message')))


    # test_positive_df = test_positive_df.sample(n=int(len(test_negative_df) * 1), replace=True, random_state=42)
    test_df = pd.concat([test_positive_df, test_negative_df]).sample(frac=1, random_state=42)

    print("\nTraining set sample count (after over-sampling positive tweets):", len(train_df))
    print("Training set positive count:", len(train_df.query("Label == 1")))
    print("Training set negative count:", len(train_df.query("Label == 0")))

    print("\nTest set sample count:", len(test_df))
    print("Test set positive count:", len(test_df.query("Label == 1")))
    print("Test set negative count:", len(test_df.query("Label == 0")))
    print("")

    # print("Training set contains positive tweets in test set:", test_positive_df['message'].isin(train_positive_df['message'].to_list()).sum())
    # print("Training set contains negative tweets in test set:", test_negative_df['message'].isin(train_negative_df['message'].to_list()).sum())
    
    print("Test set contains positive tweets in training set:", test_positive_df['message'].isin(train_positive_df['message'].to_list()).sum())
    print("Test set contains negative tweets in training set:", test_negative_df['message'].isin(train_negative_df['message'].to_list()).sum())
    
    print("Test set positive count no duplicates:", len(test_positive_df.drop_duplicates(subset='message')))
    print("Test set negative count no duplicates:", len(test_negative_df.drop_duplicates(subset='message')))

    return train_df, test_df
# make a test
train_samples_df, test_samples_df  = get_train_sample_Japanese()
test_samples_df.iloc[-5:]

# positive_df, negative_df = get_train_sample_Japanese()

e:\ProgramData\Anaconda3\envs\simple_trans_env2\lib\site-packages\ipykernel_launcher.py:226: DtypeWarning: Columns (31,50) have mixed types.Specify dtype option on import or set low_memory=False.


Positive sample count before correction: 7288
Negative sample count before correction: 23573
Total sample count beforoe correction: 30861
Need to correct tweet count: 30861
Matched tweet count: 1959
Positive sample count after correction: 7991
Negative sample count after correction: 22870
Total sample count after correction: 30861
test_positive_cnt, test_negative_cnt: 1598 9055

Positive count: 7991
Positive count no duplicates: 4980

Negative count: 22870
Negative count no duplicates: 15289

Training set sample count (before over-sampling positive tweets): 20208
Training set positive count: 6393
Training set negative count: 13815
Train set positive count no duplicates: 4195
Train set negative count no duplicates: 10715

Training set sample count (after over-sampling positive tweets): 55260
Training set positive count: 41445
Training set negative count: 13815

Test set sample count: 10653
Test set positive count: 1598
Test set negative count: 9055

Test set contains positive tweets in 

,message,Label
24133,時間は待ってくれん。ほっといたら置いてかれるぞ。死ぬ気で着いてけ。抜かしてみろよ。時間をおいてけぼりにしてみろ。,0
15332,支部やついったにアップしてくれるのも完全な好意だからお前の都合押し付けんな…無配欲しいなら借金してでも這ってでも現地に行け…死ぬ気で行け。,0
30666,【社会】「母を殺した。これから電車に飛び込む」と110番通報。男性が電車に轢かれて死亡。大田区,0
12231,死にたいー ##俺に死んで欲しい人いいね 死んでほしくない人リツイート,1
19999,もうダメだ原稿終わらない死にたい,0


In [26]:
# train_samples_df[train_samples_df['Difference'] != ""].sort_values("Difference")
# train_samples_df[train_samples_df['Difference'] != ""]["Difference"].value_counts()

In [40]:
# test_samples_df[test_samples_df['Difference'] != ""].sort_values("Difference")
# test_samples_df[test_samples_df['Difference'] != ""]["Difference"].value_counts()

In [47]:
train_unique = train_samples_df.query("Label == 1")['message'].unique()
test_unique  = test_samples_df.query("Label == 1")['message'].unique()
len(train_unique), len(test_unique)

(4195, 1320)

In [46]:
all_samples_df = pd.concat([train_samples_df, test_samples_df])
gb_df = all_samples_df.groupby(['message'], as_index=False).agg(positive_sum=("Label", 'sum'), 
                                                 label_count=("Label", 'count'))

bi_label_df = gb_df.query("label_count > 1 and positive_sum < label_count and positive_sum > 0")

print("bi_label_df label_count .sum():", bi_label_df['label_count'].sum())
bi_label_df['diff'] = bi_label_df['label_count'] - bi_label_df['positive_sum']
bi_label_df['length'] = bi_label_df['message'].str.len()
bi_label_df = bi_label_df.sort_values(['length', 'message'])
bi_label_df[['message']].to_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\bi_label_in_train.csv', index=False)
bi_label_df

bi_label_df label_count .sum(): 0


,message,positive_sum,label_count,diff,length


In [43]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model_name =  'cl-tohoku/bert-base-japanese-v2'
# model_name = r'cl-tohoku/bert-large-japanese'
# model_name = 'bert-large-uncased'

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# model = SentenceTransformer(model_name)#.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [44]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [ ]:
sweep_config = {
    "method": "random",  # grid, random, bayes
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        'train_batch_size': {'values': [8, 16, 32, 64]},
        "num_train_epochs": {"values": [2, 3, 5, 8, 10]},
        "learning_rate": {"min": 1e-5, "max": 4e-4},
    },
}

# sweep_id = wandb.sweep(sweep_config, project="Simple Sweep")
# output_dir = r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all'

def train_a_model(output_dir, train_df=train_samples_df, test_df=test_samples_df):
    
    # wandb.init()
    
    train_df['Label'] = train_df['Label'].astype(int)
    test_df['Label'] = test_df['Label'].astype(int)

    train_df = train_df.sample(frac=1)
    
    num_labels = 2
    model_args = ClassificationArgs()
    model_args.num_train_epochs = 5
    model_args.reprocess_input_data = True
    model_args.overwrite_output_dir = True
    # model_args.evaluate_during_training = True
    model_args.output_dir = output_dir
    model_args.manual_seed = 4
    model_args.use_multiprocessing = True
    model_args.train_batch_size = 8*8
    model_args.eval_batch_size = 16*8
    model_args.fp16 = True
    # model_args.labels_list = ["true", "false"]
    model_args.learning_rate = 4e-5  # 4e-6: mcc= 0.56 , 4e-4: mcc=0
    # model_args.wandb_project = "Simple Sweep"
    
    model = ClassificationModel(
    model_type = "bert",
    # model_name = r'bert-base-uncased',  
    model_name = model_name,
    # model_name = r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all_v3_verify',
        
    # model_name = r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all_v2',
    # model_name = r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all_F1_86',
    num_labels=num_labels, 
    # args={"reprocess_input_data": True,   # 对输入数据进行预处理
    #       "overwrite_output_dir": True,    # 可覆盖输出文件夹
    #       "save_steps": -1,
    #       "save_model_every_epoch": False,
    #       "output_dir": output_dir,
    #       # "train_batch_size": 1*8, # default: 8
    #       "eval_batch_size": 64*8, # default: 8
    #       },  
        
    args = model_args,
        
    
    weight=[1, 1.3],
    cuda_device=0,
        
    # sweep_config=wandb.config,
        
    )

    model.tokenizer = tokenizer

    
    model.train_model(train_df,
                      # args=model_args,
                      # args = {
                     # 'fp16':True, 
                     # "num_train_epochs": num_train_epochs,
                             # }
                      # eval_df=test_df,
                     )

    # Sync wandb
    # wandb.join()
    
    result, model_outputs, wrong_predictions = model.eval_model(test_df)
    
    
     

    print(f"In the reported positives, about {(100 * result['tp'] / (result['tp'] + result['fp'])):.0f}% are correct, i.e., {result['tp']}/({result['tp']}+{result['fp']}).")
    print(f"Precision: {(result['tp'] / (result['tp'] + result['fp'])):.3f}")
    print("")

    print(f"In the reported positives, contain about {(100 * result['tp'] / (result['fn'] + result['tp'])):.0f}% true positive tweets, i.e., {result['tp']}/({result['fn']}+{result['tp']}).")
    print(f"Recall: {(result['tp'] / (result['tp'] + result['fn'])):.3f}")

    model.save_model(output_dir=output_dir, results=result)
    
    

    return result, model_outputs, wrong_predictions


output_dir = r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all_5epochs_imbalanced_weights'
result, model_outputs, wrong_predictions = train_a_model(output_dir, train_samples_df, test_samples_df)
result
# wandb.agent(sweep_id, train_a_model)

# for idx, result in enumerate(result_list):
#     print(f"\nModel {idx + 1}")
#     print(f"In the reported positives, about {(100 * result['tp'] / (result['tp'] + result['fp'])):.0f}% are correct, i.e., {result['tp']}/({result['tp']}+{result['fp']}).")
#     print(f"Precision: {(result['tp'] / (result['tp'] + result['fp'])):.3f}")
#     print("")

#     print(f"In the reported positives, contain about {(100 * result['tp'] / (result['fn'] + result['tp'])):.0f}% true positive tweets, i.e., {result['tp']}/({result['fn']}+{result['tp']}).")
#     print(f"Recall: {(result['tp'] / (result['tp'] + result['fn'])):.3f}")

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

  0%|          | 0/55260 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_2_2


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/864 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/864 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/864 [00:00<?, ?it/s]

# result

## Test a model

In [36]:
num_labels = 2
model = ClassificationModel(
model_type = "bert",
# model_name = r'bert-base-uncased',  
# model_name = model_name,

model_name = r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all_v2',
# model_name = r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all_F1_86',
num_labels=num_labels, 
args={"reprocess_input_data": True,   # 对输入数据进行预处理
        "overwrite_output_dir": True,    # 可覆盖输出文件夹
        "save_steps": -1,
        "save_model_every_epoch": False,
        "output_dir": output_dir,
        "train_batch_size": 1*8, # default: 8
        "eval_batch_size": 64*8, # default: 8
        },  
cuda_device=0
)

model.tokenizer = tokenizer

# result, model_outputs, wrong_predictions = model.eval_model(train_samples_df)
result, model_outputs, wrong_predictions = model.eval_model(test_samples_df)

     

print(f"In the reported positives, about {(100 * result['tp'] / (result['tp'] + result['fp'])):.0f}% are correct, i.e., {result['tp']}/({result['tp']}+{result['fp']}).")
print(f"Precision: {(result['tp'] / (result['tp'] + result['fp'])):.3f}")
print("")

print(f"In the reported positives, contain about {(100 * result['tp'] / (result['fn'] + result['tp'])):.0f}% true positive tweets, i.e., {result['tp']}/({result['fn']}+{result['tp']}).")
print(f"Recall: {(result['tp'] / (result['tp'] + result['fn'])):.3f}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.
e:\ProgramData\Anaconda3\envs\simple_trans_env2\lib\site-packages\simpletransformers\classification\classification_model.py:1455: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Conver

  0%|          | 0/10653 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.8751645112440022, 'tp': 1479, 'tn': 8823, 'fp': 232, 'fn': 119, 'auroc': 0.9825329701884397, 'auprc': 0.9164437387567458, 'eval_loss': 0.29462504174028126}


In the reported positives, about 86% are correct, i.e., 1479/(1479+232).
Precision: 0.864

In the reported positives, contain about 93% true positive tweets, i.e., 1479/(119+1479).
Recall: 0.926


In [221]:
test_samples_df['Label'].sum()

1655.0

In [121]:
len(wrong_predictions)
# wrong_predictions
# np.argmax(model_outputs)
# test_samples_df[model_outputs.argmax(axis=1)]
idx = model_outputs.argmax(axis=1).astype(bool)
test_samples_df[idx].value_counts()

message                                                                                                                                                        Label
死にたい                                                                                                                                                           1        217
消えたい                                                                                                                                                           1        27 
早く死にたい                                                                                                                                                         1        27 
死にたい。                                                                                                                                                          1        12 
死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい       

In [90]:
result

{'mcc': 0.7267152341368192,
 'tp': 1792,
 'tn': 1803,
 'fp': 279,
 'fn': 290,
 'auroc': 0.9087294831228009,
 'auprc': 0.9187344927151347,
 'eval_loss': 1.0693100425932143}

# Train an existing model

In [27]:
def train_a_model(output_dir, train_df, test_df):
    train_df['Label'] = train_df['Label'].astype(int)
    test_df['Label'] = test_df['Label'].astype(int)

    train_df = train_df.sample(frac=1)
    
    num_labels = 2
    num_train_epochs = 10
    model = ClassificationModel(
    model_type = "bert",
    # model_name = r'bert-base-uncased',  
    model_name = r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all',
    # model_name = model_name, 
    num_labels=num_labels, 
    args={"reprocess_input_data": True,   # 对 输入数据进行预处理
          "overwrite_output_dir": True,    # 可覆盖输出文件夹
          "save_steps": -1,
          "save_model_every_epoch": False,
          "output_dir": output_dir,
          "train_batch_size": 1*8, # default: 8
          "eval_batch_size": 64*8, # default: 8
          },  
    weight=[1, 1.3],
    cuda_device=0,
    )

    model.tokenizer = tokenizer

    
    model.train_model(train_df,args = {'fp16':True, 
        "num_train_epochs": num_train_epochs})

    
    result, model_outputs, wrong_predictions = model.eval_model(test_df)
     

    print(f"In the reported positives, about {(100 * result['tp'] / (result['tp'] + result['fp'])):.0f}% are correct, i.e., {result['tp']}/({result['tp']}+{result['fp']}).")
    print(f"Precision: {(result['tp'] / (result['tp'] + result['fp'])):.3f}")
    print("")

    print(f"In the reported positives, contain about {(100 * result['tp'] / (result['fn'] + result['tp'])):.0f}% true positive tweets, i.e., {result['tp']}/({result['fn']}+{result['tp']}).")
    print(f"Recall: {(result['tp'] / (result['tp'] + result['fn'])):.3f}")

    model.save_model(output_dir=output_dir, results=result)

    return result, model_outputs, wrong_predictions


output_dir = r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all'
result, model_outputs, wrong_predictions = train_a_model(output_dir, train_samples_df, test_samples_df)


OSError: T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all does not appear to have a file named config.json. Checkout 'https://huggingface.co/T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all/None' for available files.

In [ ]:
import pandas as pd
df1 = pd.read_excel(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\zhenye_works\ZhenYe_FullRecord.xlsx', sheet_name='_2021-12-10T12_11_05.000Z_9526')
df2 = pd.read_excel(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\zhenye_works\ZhenYe_FullRecord.xlsx', sheet_name='_2021-12-12T12_11_05.000Z_9526')

# Predict

In [50]:
import pandas as pd

In [7]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model_name = 'cl-tohoku/bert-base-japanese-v2'
# model_name = 'bert-large-uncased'

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# model = SentenceTransformer(model_name)#.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [14]:
import glob
import random
raw_twts_files = glob.glob(os.path.join(r'T:\Shared drives\T5\Japan_Tweets_2020-01-01_2021-01-01\predicted', '*.csv'))
# raw_twts_files = glob.glob(os.path.join(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Japan_tweets\2020\cluster_csvs', '*.csv'))
raw_twts_files = sorted(raw_twts_files, reverse=True)
# raw_twts_files[-100:]
# random.sample(raw_twts_files, 3)
# raw_twts_files
print(len(raw_twts_files))
raw_twts_files.append(raw_twts_files[0])
raw_twts_files.pop(0)
raw_twts_files[0], raw_twts_files[-1]

160


('T:\\Shared drives\\T5\\Japan_Tweets_2020-01-01_2021-01-01\\predicted\\_2020-12-31T09_50_44.000Z_9649.csv',
 'T:\\Shared drives\\T5\\Japan_Tweets_2020-01-01_2021-01-01\\predicted\\positive_all.csv')

In [66]:
df = pd.read_csv(raw_twts_files[0])

In [74]:
def clean_twt_row(row, keep_emoji=False):
    try:
        cleaned = clean_twts(row['message'], keep_emoji)
        return cleaned
    except Exception as e:
        print(e, row)  
        
    

# clean_twt_row(row=df, keep_emoji=False)

In [22]:
# model_paths = [r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all_F1_86']
# model_paths = [r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all'] # Trained_model_6_japanese
model_paths = [r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Trained_model_all_20221020v1'] 

print("model_path:", model_paths)

# save_path = r'\\Desktop-h2oge6l\h\Research\Japan_tweets\predict_test'
save_path = r'T:\Shared drives\T5\Japan_Tweets_2020-01-01_2021-01-01\predicted2'

def load_trained_model(model_path):
    num_labels = 2
    # output_dir = ''
    model = ClassificationModel(
    model_type = "bert",
    # model_name = r'bert-base-uncased',  
    model_name = model_path,
    num_labels=num_labels, 
    args={"reprocess_input_data": True,   # 对输入数据进行预处理
            "overwrite_output_dir": True,    # 可覆盖输出文件夹
            "save_steps": -1,
            "save_model_every_epoch": False,     
            "eval_batch_size": 64*16,
            },  
    cuda_device=0
    )

        # model.tokonizer = tokenizer

        # basename = os.path.basename(model_path)
    return model

# def predict_by_models(df_cleaned, model):
#     results_df = df_cleaned.copy()
    

#         # model.tokonizer = tokenizer

#     basename = os.path.basename(model_path)
        
#     predictions, raw_outputs = model.predict(df_cleaned['cleaned_message'].to_list())
#     results_df[basename] = predictions
    
#     # results_df.to_csv(f"All_Model_prediction.csv", index=0)

#     return results_df



from scipy import stats

def get_final(row):
    # print(row)
    ary = row[-10:].to_list()
    final = stats.mode(ary)[0][0]
    return final

def clean_twts(tw, keep_emoji=False):
    tw = str(tw).strip()
    try:
        if tw == '':
            return tw
        # remove urls
        pattern = 'https{0,1}:\/\/t.co\/[a-zA-Z0-9]+'
        tw = re.sub(pattern, "", tw)

        # remove @  maybe no need to remove. need dig into the data.
        pattern = '@[a-zA-Z0-9_]+ '
        tw = re.sub(pattern, "", tw)


        # remove emoji
        if not keep_emoji:
            tw = emoji.demojize(tw)

        return tw

    except Exception as e:
        print(e, tw)
        
def clean_twt_row(row, keep_emoji=False):
    cleaned = clean_twts(row['message'], keep_emoji)
    return cleaned
 


    
def predict_tweet_csvs(raw_twts):    

    for idx, f in enumerate(raw_twts):
        
        # raw_df = pd.read_csv(f, engine='c')

        basename = os.path.basename(f)
        # cleaned_name = os.path.join(cleaned_path, basename)
        
        print(f"Processing: {idx + 1} / {len(raw_twts)} ", f)

        # df_cleaned = pd.read_csv(cleaned_name, engine='c')

        # print(f"Row count: {len(raw_df)}, cleaned row count: {len(df_cleaned)}" )

        df_cleaned = pd.read_csv(f, engine='python', encoding='utf-8').iloc[:]

        df_cleaned['cleaned_message'] = df_cleaned.apply(clean_twt_row, axis=1)


        df_cleaned = df_cleaned.fillna("")
        df_cleaned['message'] = df_cleaned['message'].str.strip()

        

        # empty_idxs = (df_cleaned['len'] == 0)
        # df_cleaned[empty_idxs]
        # df_cleaned.loc[empty_idxs, 'message'] = '--'

        # results_df = predict_by_models(raw_df[:], df_cleaned[:])
        # results_df = predict_by_models(df_cleaned)
        
        predictions, raw_outputs = model.predict(df_cleaned['cleaned_message'].to_list())
        
        df_cleaned['cleaned_message'] = df_cleaned.apply(lambda x: clean_twt_row(x, True), axis=1)
        
        df_cleaned['len'] = df_cleaned['message'].str.len()
        df_cleaned['len'] = df_cleaned['len'].astype(int)
        

        results_df = df_cleaned

        results_df['Label'] = predictions
        
        
            # remove the "自殺" tweets
        keywords = ['首吊り', '首を吊る', '首つり', '死ぬ気', '自分を傷つける', 'この世を去る', '死ぬに値する', '自分の人生を終わらせたいという願望', '死にたい', '自傷', '私の命を奪う', '死にたい', '死にたいです', '私の遺書', '私の人生を終わらせる', '決して起きない', '生きる価値がない', '飛び降りる', '永遠に眠る', '電車に飛び込む', '私がいないほうがいい', '生きるのに疲れた', '一人で死ぬ', '永遠に眠る', '私の悲しい人生', 'ストレスを感じる', 'ストレスで参っている', '感情の起伏が激しい', '私自身が嫌い', '精神的に弱い', '練炭', '焼身', '服毒', 'もう死にたい', '自殺サイト楽に死ねる方法', '生きることがつらい', '死にたい 助けて', '安楽死方法', '一番楽に死ねる方法', '簡単に死ねる方法', '消えたい', '確実に死ねる方法', '生きる意味が分からない', 'うつ 死にたい']
        pattern = '|'.join(keywords)  #"自殺"  "自殺"
        keywords_idx = results_df['message'].str.contains(pattern)
        sucide_idx = results_df['message'].str.contains('自殺')
        mask_idx = (~keywords_idx) & sucide_idx
        
        # return results_df
        results_df.loc[mask_idx, "Label"] = 0

        
        results_df['len'] = results_df['message'].str.len()
        # results_df['final_label'] = results_df.apply(get_final, axis=1)
        # results_df['final_label'] = results_df.iloc[:, -1]


        # results_df['final_label'] = results_df['final_label'].astype(int)
        results_df = results_df.sort_values(['Label', 'len'], ascending=[False, True])
        # results_df.to_csv(r'predictions_final_label.csv', index=False)
        # results_df.to_csv(f"All_Model_prediction.csv", index=0)

        # save_path = r'/content/drive/MyDrive/tweet_classification/annual_tweets/classified_model1'
        year = f.split('\\')[-3]
        save_path_year = os.path.join(save_path, year)
        os.makedirs(save_path_year, exist_ok=True)

        classified_name = os.path.join(save_path_year, basename)
        classified_name = classified_name[:-4] + ".csv"

        results_df.to_csv(classified_name, float_format='%.0f', index=False)

        print("Saved classified tweets in: ", classified_name)

    return results_df

model_path = model_paths[0]
model = load_trained_model(model_path)
model.tokenizer = tokenizer
results_df = predict_tweet_csvs(raw_twts_files[:])
# print("Found positive tweets:")
# results_df['Trained_model_all'].sum()

model_path: ['T:\\Shared drives\\T5\\Japan_Tweets_2021_01_01_2022_01_01\\Trained_model_all_20221020v1']


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


Processing: 1 / 160  T:\Shared drives\T5\Japan_Tweets_2020-01-01_2021-01-01\predicted\_2020-12-31T09_50_44.000Z_9649.csv


  0%|          | 0/9649 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

Saved classified tweets in:  T:\Shared drives\T5\Japan_Tweets_2020-01-01_2021-01-01\predicted2\Japan_Tweets_2020-01-01_2021-01-01\_2020-12-20T03_32_59.000Z_9568.csv
Processing: 8 / 160  T:\Shared drives\T5\Japan_Tweets_2020-01-01_2021-01-01\predicted\_2020-12-18T14_17_04.000Z_9549.csv


  0%|          | 0/9549 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [153]:
os.path.basename(os.path.join(save_path, os.path.dirname(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\Japan_tweets\2001\csv\tx.t')))

'predicted'

In [82]:
keywords = ['首吊り', '首を吊る', '首つり', '死ぬ気', '自分を傷つける', 'この世を去る', '死ぬに値する', '自分の人生を終わらせたいという願望', '死にたい', '自傷', '私の命を奪う', '死にたい', '死にたいです', '私の遺書', '私の人生を終わらせる', '決して起きない', '生きる価値がない', '飛び降りる', '永遠に眠る', '電車に飛び込む', '私がいないほうがいい', '生きるのに疲れた', '一人で死ぬ', '永遠に眠る', '私の悲しい人生', 'ストレスを感じる', 'ストレスで参っている', '感情の起伏が激しい', '私自身が嫌い', '精神的に弱い', '練炭', '焼身', '服毒', 'もう死にたい', '自殺サイト楽に死ねる方法', '生きることがつらい', '死にたい 助けて', '安楽死方法', '一番楽に死ねる方法', '簡単に死ねる方法', '消えたい', '確実に死ねる方法', '生きる意味が分からない', 'うつ 死にたい']
pattern = '|'.join(keywords)  #"自殺"  "自殺", 
# mask_ids = results_df['message'].str.contains(pattern)#.sum()
# results_df.loc[~mask_ids, 'trained_model_all'] = 0

In [128]:
pd.DataFrame([{'message':"keywords"}])['message'].str.contains(pattern)

0    False
Name: message, dtype: bool

In [102]:
train_samples_df.query("Label == 1")['message'].value_counts()
# test_samples_df.query("Label == 1")['message'].value_counts()


死にたい                                                                                                                                            2586
消えたい                                                                                                                                            331 
死にたい。                                                                                                                                           229 
早く死にたい                                                                                                                                          212 
死にたい…                                                                                                                                           116 
                                                                                                                                               ...  
中途半端にブロン飲んでもメンタル落ちるだけな気がする ちょっとは気持ちいいけど自分の無力感が上回る。すごく死にたい、いなくなりたい、生まれてこなきゃよかった、存在消してほしい 一時でも僕を必要としてくれた人だけ僕